In [ ]:
%%time
#
# Editor 2018 Takeshi Okamoto
#
# Original file
# https://github.com/thinkitcojp/TensorFlowDL-samples
# ==============================================================================
# The MIT License
# Copyright 2018 TensorFlowではじめるDeepLearning実践入門 All Rights Reserved.
# ==============================================================================
import mnistex
import tensorflow as tf

# Number of class (クラス数)
num_classes = 10

mnist = mnistex.read_data_sets(
         'train-images-idx3-ubyte', # Train images (訓練用画像)
         'train-labels-idx1-ubyte', # Train labels (訓練用ラベル)
         't10k-images-idx3-ubyte',  # Test images  (テスト用画像)
         't10k-labels-idx1-ubyte',  # Test labels  (テスト用ラベル) 
         validation_size = 5000,    # Number of Validation (検証用データ数)
                                    # "This argument can be 0" (この引数は0でも可能)
         num_classes=num_classes,   # Number of class (クラス数)
         one_hot=True               # one hot (ワンホット)
       )

''' Start model construction (モデル構築開始) '''

# Define input data (入力データを定義)
x = tf.placeholder(tf.float32, [None, 28*28])

# Shaping (整形) 28x28
img = tf.reshape(x,[-1,28,28,1])

# Convolution layer 1 (畳み込み層1) 32 channels, 28x28
f1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1))
conv1 = tf.nn.conv2d(img, f1, strides=[1,1,1,1], padding='SAME')
b1 = tf.Variable(tf.constant(0.1, shape=[32]))
h_conv1 = tf.nn.relu(conv1+b1)

# Pooling layer 1 (プーリング層1) 32 channels, 14x14
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Convolution layer 2 (畳み込み層2) 64 channels, 14x14
f2 = tf.Variable(tf.truncated_normal([3,3,32,64], stddev=0.1))
conv2 = tf.nn.conv2d(h_pool1, f2, strides=[1,1,1,1], padding='SAME')
b2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(conv2+b2)

# Pooling layer 2 (プーリング層2) 64 channels, 7x7
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Check the size of the tensor here (ここでテンソルのサイズを確認する)
print(h_pool2.get_shape());

# Convert to flat shape (フラットな形に変換)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# Total bonding layer (全結合層)
w_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1) 

# Output layer (出力層)
w_fc2 = tf.Variable(tf.truncated_normal([1024, num_classes], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[num_classes]))
out = tf.nn.softmax(tf.matmul(h_fc1, w_fc2) + b_fc2)

# Define correct type of data (正解データの型を定義)
y = tf.placeholder(tf.float32, [None, num_classes])

# Error function (誤差関数)
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 1e-5), axis=[1]))

# Training (訓練)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# Evaluation (評価)
correct = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init =tf.global_variables_initializer()

""" Execution part (実行部分)"""
with tf.Session() as sess:    
    sess.run(init)

    test_images = mnist.test.images
    test_labels = mnist.test.labels
  
    for step in range(1000):
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images ,y:train_labels})

        if step % 100 == 0:
            acc_val = sess.run( accuracy, feed_dict={x:test_images, y:test_labels})
            print('Step %d: accuracy = %.2f' % (step, acc_val))    